<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Goal" data-toc-modified-id="Goal-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Goal</a></span></li><li><span><a href="#Var" data-toc-modified-id="Var-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Var</a></span></li><li><span><a href="#Init" data-toc-modified-id="Init-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Init</a></span></li><li><span><a href="#Load" data-toc-modified-id="Load-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Load</a></span></li><li><span><a href="#SpecD" data-toc-modified-id="SpecD-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>SpecD</a></span><ul class="toc-item"><li><span><a href="#Aggregating" data-toc-modified-id="Aggregating-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Aggregating</a></span></li><li><span><a href="#Filter-by-sparsity" data-toc-modified-id="Filter-by-sparsity-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Filter by sparsity</a></span></li><li><span><a href="#Zero-estimation" data-toc-modified-id="Zero-estimation-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Zero estimation</a></span></li><li><span><a href="#clr-transformation" data-toc-modified-id="clr-transformation-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>clr transformation</a></span></li><li><span><a href="#New-phyloseq-obj." data-toc-modified-id="New-phyloseq-obj.-5.5"><span class="toc-item-num">5.5&nbsp;&nbsp;</span>New phyloseq obj.</a></span></li></ul></li><li><span><a href="#IndD" data-toc-modified-id="IndD-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>IndD</a></span><ul class="toc-item"><li><span><a href="#Aggregating" data-toc-modified-id="Aggregating-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Aggregating</a></span></li><li><span><a href="#Filter-by-sparsity" data-toc-modified-id="Filter-by-sparsity-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Filter by sparsity</a></span></li><li><span><a href="#Zero-estimation" data-toc-modified-id="Zero-estimation-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Zero estimation</a></span></li><li><span><a href="#clr-transformation" data-toc-modified-id="clr-transformation-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>clr transformation</a></span></li><li><span><a href="#New-phyloseq-obj" data-toc-modified-id="New-phyloseq-obj-6.5"><span class="toc-item-num">6.5&nbsp;&nbsp;</span>New phyloseq obj</a></span></li></ul></li><li><span><a href="#sessionInfo" data-toc-modified-id="sessionInfo-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>sessionInfo</a></span></li></ul></div>

# Goal

* Create coda-transformed dataset
  * accounting for compositional data
* Method
  * impute zeros with zCompositions
  * perform clr transformation on counts
    * `compositions::clr`

# Var

In [1]:
work_dir = '/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/'

# microbiome
## 16S-arch phyloseq object (one sample per species)
physeq_arch_SpecD_file = '/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/phyloseq/physeq_arch_SpecD.RDS'
## 16S-arch phyloseq object (one sample per individual)
physeq_arch_IndD_file = '/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/phyloseq/physeq_arch_IndD.RDS'

# params
threads = 8
my_seed = 8371


# Init

In [2]:
library(dplyr)
library(tidyr)
library(ggplot2)

library(phyloseq)
library(zCompositions)

library(LeyLabRMisc)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: MASS


Attaching package: ‘MASS’


The following object is masked from ‘package:dplyr’:

    select


Loading required package: NADA

Loading required package: survival


Attaching package: ‘NADA’


The following object is masked from ‘package:stats’:

    cor


Loading required package: truncnorm



In [3]:
df.dims()
set.seed(my_seed)
make_dir(work_dir)

Directory already exists: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/ 


# Load

In [4]:
# phyloseq
physeq_arch_SpecD = readRDS(physeq_arch_SpecD_file)
physeq_arch_SpecD

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 1151 taxa and 110 samples ]
sample_data() Sample Data:       [ 110 samples by 88 sample variables ]
tax_table()   Taxonomy Table:    [ 1151 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 1151 tips and 1150 internal nodes ]

In [5]:
# phyloseq
physeq_arch_IndD = readRDS(physeq_arch_IndD_file)
physeq_arch_IndD

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 1891 taxa and 185 samples ]
sample_data() Sample Data:       [ 185 samples by 88 sample variables ]
tax_table()   Taxonomy Table:    [ 1891 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 1891 tips and 1888 internal nodes ]

# SpecD

## Aggregating

In [9]:
physeq_arch_SpecD = tax_glom(physeq_arch_SpecD, taxrank='Class')
physeq_arch_SpecD

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 10 taxa and 110 samples ]
sample_data() Sample Data:       [ 110 samples by 88 sample variables ]
tax_table()   Taxonomy Table:    [ 10 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 10 tips and 9 internal nodes ]

## Filter by sparsity

In [10]:
# filter by sparsity
physeq_arch_SpecD = physeq_arch_SpecD %>%
    filter_taxa(function(x) sum(x > 0) >= 2, TRUE) %>%
    prune_samples(sample_sums(.) > 0, .)
physeq_arch_SpecD

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 9 taxa and 110 samples ]
sample_data() Sample Data:       [ 110 samples by 88 sample variables ]
tax_table()   Taxonomy Table:    [ 9 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 9 tips and 8 internal nodes ]

In [11]:
# otu table
otu = physeq_arch_SpecD %>%
    phyloseq2df(otu_table) %>%
    as.data.frame
rownames(otu) = otu$OTU
otu$OTU = NULL
otu = otu %>% t %>% as.data.frame
otu

,cd161a55fa07f62886e28ad1665f4153,5887dc080d027d30580310a1eb1f3166,dc2d500b34b0dc2adb147b190d3b337a,00e469ce4b434d2939579272171847d0,e569d58f2813d042dd490d75c6149977,6656e50d37ef6857fdcf857d1f735353,2987f396047df01622b365b3b667a4eb,75f3ea1df98051a1b49107173aa51a32,1578a874797e43b052a66b2fb5ef9e4a
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
F14_Common_Bream,0,2,0,812,0,14,16,0,2613
F44_Fallow_Deer,0,0,0,0,0,0,570,0,5767
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
X405_Nile_Crocodile,0,3,10,51,375,16,0,0,30
X67_European_Roe,0,0,0,0,0,0,0,0,721


## Zero estimation

In [12]:
# sparsity (%)
otu %>%
    apply(2, function(x) sum(x == 0) / length(x) * 100) %>%
    summary_x('sparsity')

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
sparsity,2.727273,51.81818,92.72727,71.41414,94.54545,98.18182,36.767,15.01


In [13]:
# inferring zeros
## warning: a bit slow
otu = cmultRepl(otu, output = 'p-counts')
otu

No. corrected values:  237 


,cd161a55fa07f62886e28ad1665f4153,5887dc080d027d30580310a1eb1f3166,dc2d500b34b0dc2adb147b190d3b337a,00e469ce4b434d2939579272171847d0,e569d58f2813d042dd490d75c6149977,6656e50d37ef6857fdcf857d1f735353,2987f396047df01622b365b3b667a4eb,75f3ea1df98051a1b49107173aa51a32,1578a874797e43b052a66b2fb5ef9e4a
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
F14_Common_Bream,0.009311928,2.00000000,1.789753,812.000000,1.391459,14.00000000,16,0.2619756,2613
F44_Fallow_Deer,0.009374887,0.04562445,1.801853,5.020784,2.552064,0.09249889,570,0.2637468,5767
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
X405_Nile_Crocodile,0.007455791,3.00000000,10.0000000,51.0000000,375.0000000,16.00000000,0.01953328,0.2097563,30
X67_European_Roe,0.007933819,0.03861125,0.5657115,0.5717932,0.2906424,0.07828034,0.02909664,0.2232048,721


In [14]:
# check
## sparsity (%)
otu %>%
    apply(1, function(x) sum(x == 0) / length(x) * 100) %>%
    summary_x('sparsity')

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
sparsity,0,0,0,0,0,0,0,0


## clr transformation

In [15]:
# transforming
otu = compositions::clr(otu) %>%
    as.data.frame
otu

,cd161a55fa07f62886e28ad1665f4153,5887dc080d027d30580310a1eb1f3166,dc2d500b34b0dc2adb147b190d3b337a,00e469ce4b434d2939579272171847d0,e569d58f2813d042dd490d75c6149977,6656e50d37ef6857fdcf857d1f735353,2987f396047df01622b365b3b667a4eb,75f3ea1df98051a1b49107173aa51a32,1578a874797e43b052a66b2fb5ef9e4a
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
F14_Common_Bream,-6.406350,-1.036743,-1.1478132,4.969610,-1.3995375,0.9091667,1.042698,-3.069395,6.138364
F44_Fallow_Deer,-5.411331,-3.828922,-0.1527944,0.871976,0.1952922,-3.1221688,5.604026,-2.074376,7.918297
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
X405_Nile_Crocodile,-5.902934,0.0944427,1.2984155,2.9276560,4.92275643,1.768419,-4.939805,-2.5659787,2.397028
X67_European_Roe,-3.563425,-1.9810159,0.7035247,0.7142178,0.03753398,-1.274263,-2.263937,-0.2264701,7.853835


In [16]:
# summary
otu %>% rowSums %>% summary
otu %>% colSums %>% summary

      Min.    1st Qu.     Median       Mean    3rd Qu.       Max. 
-4.885e-15 -8.188e-16  9.714e-17 -2.422e-17  8.882e-16  3.886e-15 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-501.48 -241.81   28.45    0.00   99.93  738.70 

## New phyloseq obj.

In [17]:
# new phyloseq object
physeq_arch_SpecD_clr = phyloseq(otu_table(otu %>% t %>% as.data.frame, 
                                           taxa_are_rows=TRUE),
                                tax_table(physeq_arch_SpecD),
                                sample_data(physeq_arch_SpecD),
                                phy_tree(physeq_arch_SpecD))
physeq_arch_SpecD_clr

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 9 taxa and 110 samples ]
sample_data() Sample Data:       [ 110 samples by 88 sample variables ]
tax_table()   Taxonomy Table:    [ 9 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 9 tips and 8 internal nodes ]

In [18]:
# writing new file
physeq_arch_SpecD_clr_file = file.path(dirname(physeq_arch_SpecD_file),
                                      'physeq_arch_SpecD_clr-class.RDS')
saveRDS(physeq_arch_SpecD_clr, physeq_arch_SpecD_clr_file)
cat('File written:', physeq_arch_SpecD_clr_file, '\n')

File written: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/phyloseq/physeq_arch_SpecD_clr-class.RDS 


# IndD

## Aggregating

In [19]:
physeq_arch_IndD = tax_glom(physeq_arch_IndD, taxrank='Class')
physeq_arch_IndD

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 11 taxa and 185 samples ]
sample_data() Sample Data:       [ 185 samples by 88 sample variables ]
tax_table()   Taxonomy Table:    [ 11 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 11 tips and 10 internal nodes ]

## Filter by sparsity

In [20]:
# filter by sparsity
physeq_arch_IndD = physeq_arch_IndD %>%
    filter_taxa(function(x) sum(x > 0) >= 2, TRUE) %>%
    prune_samples(sample_sums(.) > 0, .)
physeq_arch_IndD

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 9 taxa and 185 samples ]
sample_data() Sample Data:       [ 185 samples by 88 sample variables ]
tax_table()   Taxonomy Table:    [ 9 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 9 tips and 8 internal nodes ]

In [21]:
# otu table
otu = physeq_arch_IndD %>%
    phyloseq2df(otu_table) %>%
    as.data.frame
rownames(otu) = otu$OTU
otu$OTU = NULL
otu = otu %>% t %>% as.data.frame
otu

,cd161a55fa07f62886e28ad1665f4153,5887dc080d027d30580310a1eb1f3166,861c85239d21bf467158a1a867489ebf,16c9cf055840535f82770aa57f56486a,e569d58f2813d042dd490d75c6149977,f7b992dfabd26d5316abbe75dba81cad,2987f396047df01622b365b3b667a4eb,75f3ea1df98051a1b49107173aa51a32,1578a874797e43b052a66b2fb5ef9e4a
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
F14_Common_Bream,0,2,0,812,0,14,16,0,2613
F35_Red_Deer,0,4,0,554,0,0,81,0,1235
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
X431_Quagga,0,0,0,2136,0,0,267,0,30
X67_European_Roe,0,0,0,0,0,0,0,0,721


## Zero estimation

In [22]:
# sparsity (%)
otu %>%
    apply(1, function(x) sum(x == 0) / length(x) * 100) %>%
    summary_x('sparsity')

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
sparsity,33.33333,66.66667,66.66667,70.63063,77.77778,88.88889,18.663,7.619


In [23]:
# inferring zeros
## warning: a bit slow
otu = cmultRepl(otu, output = 'p-counts')
otu

No. corrected values:  451 


,cd161a55fa07f62886e28ad1665f4153,5887dc080d027d30580310a1eb1f3166,861c85239d21bf467158a1a867489ebf,16c9cf055840535f82770aa57f56486a,e569d58f2813d042dd490d75c6149977,f7b992dfabd26d5316abbe75dba81cad,2987f396047df01622b365b3b667a4eb,75f3ea1df98051a1b49107173aa51a32,1578a874797e43b052a66b2fb5ef9e4a
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
F14_Common_Bream,0.004328734,2,1.0232501,812,1.1065906,14.0000000,16,0.2383689,2613
F35_Red_Deer,0.004191709,4,0.5547652,554,0.5999491,0.1433565,81,0.2308235,1235
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
X431_Quagga,0.004255424,0.03546187,0.7202262,2136.0000000,0.7788864,0.1455355,267.00000000,0.2343320,30
X67_European_Roe,0.003783021,0.03152518,0.2136216,0.3585286,0.2310204,0.1293793,0.02907248,0.2083184,721


## clr transformation

In [24]:
# transforming
otu = compositions::clr(t(otu)) %>%
    t %>% as.data.frame
otu

,cd161a55fa07f62886e28ad1665f4153,5887dc080d027d30580310a1eb1f3166,861c85239d21bf467158a1a867489ebf,16c9cf055840535f82770aa57f56486a,e569d58f2813d042dd490d75c6149977,f7b992dfabd26d5316abbe75dba81cad,2987f396047df01622b365b3b667a4eb,75f3ea1df98051a1b49107173aa51a32,1578a874797e43b052a66b2fb5ef9e4a
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
F14_Common_Bream,-0.00545212,3.774822,0.3864814,4.189274,-0.4028117,4.4313581,1.826603,-0.1864763,1.1855663
F35_Red_Deer,-0.03761856,4.467969,-0.2257129,3.806939,-1.0150060,-0.1501203,3.448463,-0.2186428,0.4361383
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
X431_Quagga,-0.02253269,-0.2576229,0.03530752,5.156464,-0.7539856,-0.1350344,4.641263,-0.2035569,-3.2814906
X67_European_Roe,-0.14020423,-0.3752945,-1.18005175,-3.535973,-1.9693448,-0.2527060,-4.483949,-0.3212284,-0.1020488


In [25]:
# summary
otu %>% rowSums %>% summary
otu %>% colSums %>% summary

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-18.083  -5.477   1.318   0.000   5.846  19.795 

      Min.    1st Qu.     Median       Mean    3rd Qu.       Max. 
-1.270e-13 -1.760e-14  2.731e-14  6.742e-14  7.039e-14  6.128e-13 

## New phyloseq obj

In [26]:
# new phyloseq object
physeq_arch_IndD_clr = phyloseq(otu_table(otu %>% t %>% as.data.frame, 
                                          taxa_are_rows=TRUE),
                                tax_table(physeq_arch_IndD),
                                sample_data(physeq_arch_IndD),
                                phy_tree(physeq_arch_IndD))
physeq_arch_IndD_clr

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 9 taxa and 185 samples ]
sample_data() Sample Data:       [ 185 samples by 88 sample variables ]
tax_table()   Taxonomy Table:    [ 9 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 9 tips and 8 internal nodes ]

In [27]:
# writing new file
physeq_arch_IndD_clr_file = file.path(dirname(physeq_arch_IndD_file),
                                      'physeq_arch_IndD_clr-class.RDS')
saveRDS(physeq_arch_IndD_clr, physeq_arch_IndD_clr_file)
cat('File written:', physeq_arch_IndD_clr_file, '\n')

File written: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/phyloseq/physeq_arch_IndD_clr-class.RDS 


# sessionInfo

In [28]:
sessionInfo()

R version 3.6.2 (2019-12-12)
Platform: x86_64-conda_cos6-linux-gnu (64-bit)
Running under: Ubuntu 18.04.4 LTS

Matrix products: default
BLAS/LAPACK: /ebio/abt3_projects/Georg_animal_feces/envs/phyloseq-eco/lib/libopenblasp-r0.3.7.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] LeyLabRMisc_0.1.6   zCompositions_1.3.4 truncnorm_1.0-8    
 [4] NADA_1.6-1.1        survival_3.1-8      MASS_7.3-51.5      
 [7] phyloseq_1.30.0     ggplot2_3.2.1       tidyr_1.0.0        
[10] dplyr_0.8.5        

loaded via a namespace (and not attached):
 [1] Biobase_2.